# Методы машинного обучения – Лабораторная работа №5

# Рекуррентные нейронные сети RNN

Импортируем необходимые библиотеки:

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
tf.__version__

## Задача аппроксимации синусоиды при помощи сети MLP

В нейронных сетях прямого распространения (многослойных перцептронах, сетях MLP) поток данных движется только в одном направлении, а именно от входного слоя к выходному через скрытые слои.

Рассмотрим сеть прямого распространения (сеть MLP) с одним скрытым слоем, использующую нелинейную функцию активации, чтобы распознать нелинейную зависимость, задаваемую синусоидой. 

Будем использовать сеть MLP с одним входным нейроном, десятью скрытыми нейронами и одним выходным нейроном. Скрытые нейроны используют функцию активации гиперболический тангенс ($\tanh$), тогда как выходной слой использует тождественную функцию активации. 

In [ ]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(10, input_shape=(1,), activation='tanh', name='HiddenLayer'),  
  tf.keras.layers.Dense(1, name='OutputLayer')])

In [ ]:
model.summary()

При обучении нейронной сети будем использовать оптимизатор `rmsprop` (применяется по умолчанию). В качестве функции ошибок (потерь) будем использовать среднеквадратичную ошибку MSE (mean squared error), а для оценки качества модели – среднюю абсолютную ошибку MAE (mean absolute error).

In [ ]:
model.compile(optimizer="rmsprop", # по умолчанию
              loss="mse",
              metrics=["mae"])

Обучающие данные содержат $50$ точек $x_{i}$, выбранных случайным образом в диапазоне $\left[-10,\,10\right]$, где $y_{i}=\sin\left(x_{i}\right)$. 

In [ ]:
x_train = 20 * np.random.random(50) - 10
y_train = np.sin(x_train)

x_plot = np.linspace(-10,10,101)
y_plot = np.sin(x_plot)

In [ ]:
mpl.rcParams['figure.figsize'] = (12,8)

plt.plot(x_plot, y_plot, c='b')
plt.scatter(x_train, y_train, s=100, c='r', marker='*');

Будем обучать нейронную сеть на сгенерированных 50 точках, увеличивая количество эпох обучения:

In [ ]:
model.fit(x_train, y_train, epochs=1000, verbose=0);

Оценим качество полученной модели на множестве равноотстоящих точек из отрезка $[-10,10]$:

In [ ]:
x_pred = np.linspace(-10,10,1001)
y_pred = model.predict(x_pred)

Для визуализации прогноза модели будем использовать функцию:

In [ ]:
def show_sin():
    print('MSE: {:.3f}, MAE:{:.3f}'.format(*model.evaluate(x_plot, y_plot, verbose=0)))

    plt.plot(x_plot, y_plot, c='b')
    plt.scatter(x_train, y_train, s=100, c='r', marker='*')
    plt.plot(x_pred, y_pred, c='g', lw=3, alpha=0.5);
    
show_sin()

Увеличим количество эпох обучения до десяти тысяч:

In [ ]:
model.fit(x_train, y_train, initial_epoch=1000, epochs=10000, verbose=0);
y_pred = model.predict(x_pred)
show_sin()

До двацати тысяч:

In [ ]:
model.fit(x_train, y_train, initial_epoch=10000, epochs=20000, verbose=0);
y_pred = model.predict(x_pred)
show_sin()

И, наконец, до тридцати тысяч:

In [ ]:
model.fit(x_train, y_train, initial_epoch=20000, epochs=30000, verbose=0);
y_pred = model.predict(x_pred)
show_sin()

Рисунок свидетельствует о хорошем качестве аппроксимации синусоиды нейронной сетью.

Однако выполним прогноз для значений из расширенного диапазона $[-20,20]$ и нарисуем кривые:

In [ ]:
x_pred = np.linspace(-20,20,1001)
y_pred = model.predict(x_pred)

show_sin()

Таким образом, фактически нейронная сеть прямого распространения только аппроксимировала синусоиду на интервале $[-10,10]$ и не выявила периодичность аппроксимируемой функции. 

## Задача прогнозирования поведения синусоиды при помощи сети MLP

Рассмотрим теперь задачу прогнозирования значений синусоиды.

Будем рассматривать в качестве исходного датасета набор значений синусоиды на интервале от $[-10,10]$.

In [ ]:
ds_data = np.sin(np.linspace(-10,10,1001))

Обучающая выборка составляет 80% всех данных:

In [ ]:
train_size = int(len(ds_data) * 0.8)
test_size = len(ds_data) - train_size
ds_train, ds_test = ds_data[:train_size], ds_data[train_size:]
ds_train.shape, ds_test.shape

Конвертируем исходный набор в нужный формат:

In [ ]:
def create_ds(ds, look_back=1):
  dataX, dataY = [], []
  for i in range(len(ds)-look_back-1):
    a = ds[i:(i+look_back)]
    dataX.append(a)
    dataY.append(ds[i + look_back])
  return np.array(dataX), np.array(dataY)

Будем делать прогноз на основе 10 предыдущих значений:

In [ ]:
look_back = 10
trainX, trainY = create_ds(ds_train, look_back=look_back)
testX, testY = create_ds(ds_test, look_back=look_back)
trainX.shape, trainY.shape, testX.shape, testY.shape

Изменим форму обучающего и тестового наборов данных:

In [ ]:
# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
trainX.shape, testX.shape

Создадим сеть MLP с одним скрытым слоем:

In [ ]:
model_mlp = tf.keras.Sequential([
  tf.keras.layers.Dense(10, input_shape=(1,look_back), activation='tanh', name='HiddenLayer'),  
  tf.keras.layers.Dense(1, name='OutputLayer')])
model_mlp.summary()

In [ ]:
model_mlp.compile(loss='mean_squared_error', optimizer='adam')
model_mlp.fit(trainX, trainY, epochs=100, batch_size=128, verbose=2)

Выполним прогноз:

In [ ]:
trainPredict = model_mlp.predict(trainX)
testPredict = model_mlp.predict(testX)

Используем для оценки качества показатель RMSE, вычисляемый через MSE:

In [ ]:
def my_mse(y_test, y_predict):
    return np.sum((y_predict - y_test)**2) / len(y_test)

In [ ]:
trainScore = np.sqrt(my_mse(trainY, trainPredict.reshape(-1)))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = np.sqrt(my_mse(testY, testPredict.reshape(-1)))
print('Test Score: %.2f RMSE' % (testScore))

Для корректной визуализации данных нужно сдвинуть данные:

In [ ]:
# shift train predictions for plotting
trainPredictPlot = np.empty_like(ds_data)
trainPredictPlot[:] = np.nan
trainPredictPlot[look_back:len(trainPredict) + look_back] = trainPredict.reshape(-1)

In [ ]:
# shift test predictions for plotting
testPredictPlot = np.empty_like(ds_data)
testPredictPlot[:,] = np.nan
testPredictPlot[len(trainPredict) + (look_back * 2) + 1:len(ds_data) - 1] = testPredict.reshape(-1)

In [ ]:
plt.plot(ds_data, label='Actual')
plt.plot(trainPredictPlot.reshape(-1), label='Training')
plt.plot(testPredictPlot.reshape(-1), label='Testing')
plt.legend(loc='best');

### Решение проблемы исчезающих градиентов

#### Инициализация весов

Для управления инициализацией весов нужно указать при создании слоя параметр `kernel_initializer`, например, так:

`tf.keras.layers.Dense(10, activation = "relu", kernel_initializer="he_normal")`

или

`tf.keras.layers.Dense(10, activation = "relu", kernel_initializer="he_uniform")`

или

`tf.keras.layers.Dense(10, activation = "sigmoid", kernel_initializer=keras.initializers.VarianceScaling(scale=2., mode='fan_avg', distribution='uniform'))`

#### Использование функций активации без насыщения

Например, можно использовать функцию активации `SELU` (Scaled Exponential Linear Unit) с инициализацией весов `lecun_normal` так:

`model.add(tf.keras.layers.Dense(10, kernel_initializer='lecun_normal',
                                 activation='selu'))`

#### Пакетная нормализация (Batch Normalization)

Слой пакетной нормализации может быть добавлен в модель так:

`model.add(tf.keras.layers.BatchNormalization())`

#### Обрезка градиентов (Gradient Clipping)

При обрезке градентов при задании оптимизатора в методе `compile()` указывается параметр `clipvalue` или `clipnorm`, например: 

`model.compile(optimizer=optimizer = keras.optimizers.SGD(clipvalue = 1.0),
              loss="mse", metrics=["mae"])`   

или

`model.compile(optimizer=optimizer = keras.optimizers.SGD(clipnorm = 1.0),
              loss="mse", metrics=["mae"])` 

Попробуем решить задачу прогноза значений синусоиды при помощи глубокой сети MLP с применением пакетной нормализации, функций активации без насыщения и альтернативной инициализации весов:

In [ ]:
model_dmlp = tf.keras.Sequential([
    tf.keras.layers.Dense(10, input_shape=(1,look_back), 
                          kernel_initializer='lecun_normal', activation='selu'),  
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10, kernel_initializer='lecun_normal', activation='selu'),  
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(1, name='OutputLayer')])
model_dmlp.summary()

In [ ]:
model_dmlp.compile(loss='mean_squared_error', optimizer='adam')
model_dmlp.fit(trainX, trainY, epochs=100, batch_size=128, verbose=2)

In [ ]:
trainPredict = model_dmlp.predict(trainX)
testPredict = model_dmlp.predict(testX)

In [ ]:
trainScore = np.sqrt(my_mse(trainY, trainPredict.reshape(-1)))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = np.sqrt(my_mse(testY, testPredict.reshape(-1)))
print('Test Score: %.2f RMSE' % (testScore))

## Рекуррентная нейронная сеть RNN

__Рекуррентные нейронные сети__ (Recurrent Neural Networks, __RNN__) включают обратную связь от одного уровня к другому, и их обычно можно обучить, развертывая рекуррентные соединения, в результате чего получаются глубокие сети, параметры которых можно обучить с помощью алгоритма обратного распространения ошибки. 

Если многослойные персептроны представляют собой сети прямого распространения, в которых информация движется только в одном направлении, а именно от входного слоя к выходному через скрытые слои, то рекуррентные нейронные сети (RNN) содержат петли (циклы) обратной связи между двумя (или более) слоями, что делает их идеальными для обучения на основе входных данных в форме последовательностей. 

Задача сети RNN состоит в том, чтобы аппроксимировать функцию, которая предсказывает целевую выходную последовательность $\mathcal{Y}$ по заданной входной последовательности $\mathcal{X}$. То есть прогнозируемые выходные данные (результат) $\mathbf{o}_{t}$ для входных данных $\mathbf{x}_{t}$ должны быть аналогичны или близки к целевому отклику $\mathbf{y}_{t}$ для каждого момента времени $t$.

In [ ]:
model_rnn = tf.keras.Sequential([
  tf.keras.layers.SimpleRNN(10, input_shape=(1,look_back), name='HiddenLayer'),  
  tf.keras.layers.Dense(1, name='OutputLayer')])
model_rnn.summary()

In [ ]:
model_rnn.compile(loss='mean_squared_error', optimizer='adam')
model_rnn.fit(trainX, trainY, epochs=100, batch_size=128, verbose=2)

In [ ]:
trainPredict = model_rnn.predict(trainX)
testPredict = model_rnn.predict(testX)

In [ ]:
trainScore = np.sqrt(my_mse(trainY, trainPredict.reshape(-1)))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = np.sqrt(my_mse(testY, testPredict.reshape(-1)))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
# shift train predictions for plotting
trainPredictPlot = np.empty_like(ds_data)
trainPredictPlot[:] = np.nan
trainPredictPlot[look_back:len(trainPredict) + look_back] = trainPredict.reshape(-1)

In [ ]:
# shift test predictions for plotting
testPredictPlot = np.empty_like(ds_data)
testPredictPlot[:,] = np.nan
testPredictPlot[len(trainPredict) + (look_back * 2) + 1:len(ds_data) - 1] = testPredict.reshape(-1)

In [ ]:
plt.plot(ds_data, marker='o', c='y', lw=5, label='Actual')
plt.plot(trainPredictPlot.reshape(-1), c='b', label='Training')
plt.plot(testPredictPlot.reshape(-1), c='r', label='Testing')
plt.legend(loc='best');

Рассмотрим теперь более общую задачу прогнозирования значений временного ряда.

## Прогнозирование значений временного ряда

_Временной ряд_ представляет собой любые данные, полученные путем измерений через равные промежутки времени, например, дневная цена акции, почасовое потребление электроэнергии в городе или еженедельные продажи в магазине. Временные ряды встречаются в различных сферах  деятельности от изучения природных явлений (сейсмическую активность, эволюция популяций рыб в реке, погода в определенном месте и пр.) до экономики (посетители веб-сайта, ВВП страны, операции с кредитными картами и т.п.).

Наиболее распространенной задачей, связанной с временными рядами, является прогнозирование – предсказание дальнейшей динамики временного ряда. Необходимо прогнозировать потребление электроэнергии, чтобы предвидеть спрос; прогнозировать доход на несколько месяцев вперед, чтобы планировать бюджет; прогнозировать погоду на несколько дней вперед, чтобы планировать расписание.

Рассмотрим задачу прогнозирования температуры воздуха через 24 часа, учитывая временной ряд часовых измерений таких величин, как атмосферное давление и влажность. 

Этот набор данных был записан на метеостанции в Институте биогеохимии им. Макса Планка в Йене, Германия. В этом наборе 14 различных величин (таких как температура, давление, влажность, направление ветра и т.д.) записанных с интервалом 10 минут за несколько лет. Набор можно загрузить по адресу `https://s3.amazonaws.com/keras-datasets/jena_climate_2009_2016.csv.zip`.

In [ ]:
import os
fname = os.path.join("jena_climate_2009_2016.csv")

with open(fname) as f:
    data = f.read()

lines = data.split("\n")      # list with headers and text lines with date and data
header = lines[0].split(",")  # list of column names
lines = lines[1:]             # list of text lines with date and data
print('Атрибуты набора данных:', header)
print('\nКоличество записей:', len(lines))

Всего в наборе 420451 строк данных (записей), содержащих метку с датой/временем и 14 погодных показателей. Исключим столбец с датой/временем и загрузим температурные показатели в массив `temperature` и все показатели (включая температуру) в массив `raw_data`. 

Нарисуем также график температуры (в градусах Цельсия) в зависимости от времени (номера записи):

In [ ]:
temperature = np.zeros((len(lines),))
raw_data = np.zeros((len(lines), len(header) - 1))
for i, line in enumerate(lines):
    values = [float(x) for x in line.split(",")[1:]] # list of values w/o date
    temperature[i] = values[1]                       # temperature only
    raw_data[i, :] = values[:]                       # all values
    
plt.plot(range(len(temperature)), temperature);

На графике прослеживаются сезонные изменения температуры (за 8 лет). Однако температура также изменяется в зависимости от времени суток. Это видно на температурном графике за 10 дней (т.к. данные измеряются каждые 10 минут, ежедневно сохраняется $24 * 6 = 144$ записи).

In [ ]:
plt.plot(range(1440), temperature[:1440]); # 1440 = 10 x 24 x 6 

Будем использовать первые 50% данных в наборе для обучения, следующие 25% для валидации и последние 25% для тестирования. Вычислим количество записей в обучающей, валидационной и тестовой выборках: 

In [ ]:
num_train_samples = int(0.5 * len(raw_data))
num_val_samples = int(0.25 * len(raw_data))
num_test_samples = len(raw_data) - num_train_samples - num_val_samples
print("num_train_samples:", num_train_samples)
print("num_val_samples:", num_val_samples)
print("num_test_samples:", num_test_samples)

Нормализуем каждый столбец в массиве `raw_data` по данным обучающей выборки (по первым `num_train_samples` записям), чтобы все столбцы принимали небольшие значения в одинаковом масштабе:

In [ ]:
mean = raw_data[:num_train_samples].mean(axis=0)
raw_data -= mean
std = raw_data[:num_train_samples].std(axis=0)
raw_data /= std # whole raw_data normalized w.r.t. first num_train_samples rows

mean.shape, std.shape

Теперь создадим набор (объект) с данными за последние пять дней вместе с целевым показателем температуры через 24 часа. Для этого используем встроенную в модуль Keras функцию `timeseries_dataset_from_array()`. Чтобы проиллюстрировать работу этой функции, рассмотрим следующий простой пример:

In [ ]:
int_sequence = np.arange(10)   # [0,1,2,3,4,5,6,7,8,9]
dummy_dataset = tf.keras.utils.timeseries_dataset_from_array( # объект BatchDataset
    data=int_sequence[:-3],    # [0,1,2,3,4,5,6]
    targets=int_sequence[3:],  # [3,4,5,6,7,8,9]
    sequence_length=3,         # длина последовательности
    batch_size=2,              # размер батча (пакета)
)

for inputs, targets in dummy_dataset: # tf.Tensor objects
    for i in range(inputs.shape[0]):
        print([int(x) for x in inputs[i]], int(targets[i]))
    print("-"*10)

Используем `timeseries_dataset_from_array()` для создания трех наборов данных (объектов `BatchDataset`) – для обучения, для валидации и для тестирования со следующими значениями параметров:

* `sample_rate = 6` — наблюдения будут производиться в одной точке данных в час - будем хранить только одну точку данных из 6;
* `sequence_length=120` — наблюдения будут браться за 5 дней (120 часов);
* `delay=sampling_rate*(sequence_length+24-1)` — целью для входной последовательности будет температура через 24 часа после окончания последовательности;

При создании набора обучающих данных передадим в функцию аргементы `start_index = 0` и `end_index = num_train_samples`, чтобы использовать только первые 50% данных. Для набора данных валидации передадим в функцию `start_index = num_train_samples` и `end_index = num_train_samples + num_val_samples`, чтобы использовать следующие 25% данных. Наконец, для тестового набора данных передадим `start_index=num_train_samples+num_val_samples`, чтобы использовать оставшиеся данные.

In [ ]:
sampling_rate = 6
sequence_length = 120
delay = sampling_rate * (sequence_length + 24 - 1)
batch_size = 256

train_dataset = tf.keras.utils.timeseries_dataset_from_array(
    raw_data[:-delay],
    targets=temperature[delay:],
    sampling_rate=sampling_rate,
    sequence_length=sequence_length,
    shuffle=True,
    batch_size=batch_size,
    start_index=0,
    end_index=num_train_samples)

val_dataset = tf.keras.utils.timeseries_dataset_from_array(
    raw_data[:-delay],
    targets=temperature[delay:],
    sampling_rate=sampling_rate,
    sequence_length=sequence_length,
    shuffle=True,
    batch_size=batch_size,
    start_index=num_train_samples,
    end_index=num_train_samples + num_val_samples)

test_dataset = tf.keras.utils.timeseries_dataset_from_array(
    raw_data[:-delay],
    targets=temperature[delay:],
    sampling_rate=sampling_rate,
    sequence_length=sequence_length,
    shuffle=True,
    batch_size=batch_size,
    start_index=num_train_samples + num_val_samples)

Каждый набор данных содержит кортежи `(samples, targets)`, где `samples` представляет собой пакет из 256 записей, каждая из которых содержит 120 последовательных часовых записей данных, а `targets` представляет собой соответствующий массив из 256 температур. Обратите внимание, что записи перемешиваются случайным образом, поэтому две соседние последовательности в пакете (например, `samples[0]` и `samples[1]`) не обязательно близки по времени.

In [ ]:
for samples, targets in train_dataset: # shapes in first data batch
    print("Форма признаков:", samples.shape)
    print("Форма откликов:", targets.shape)
    break # first batch only

Оценим ошибку для следующего элементарного подхода к прогнозированию температуры, когда в качестве прогнозного значения температуры через 24 часа принимается текущее значение температуры. В качестве показателя качества модели прогнозирования будем использовать метрику MAE (mean absolute error):

In [ ]:
def evaluate_naive_method(dataset):
    total_abs_err = 0.
    samples_seen = 0
    for samples, targets in dataset:
        preds = samples[:, -1, 1] * std[1] + mean[1] # std and mean of temperature
        total_abs_err += np.sum(np.abs(preds - targets))
        samples_seen += samples.shape[0]
    return total_abs_err / samples_seen

print(f"MAE на валидационном наборе: {evaluate_naive_method(val_dataset):.2f}")
print(f"MAE на тестовом наборе: {evaluate_naive_method(test_dataset):.2f}")

## Прогнозирование температуры при помощи сети MLP

Рассмотрим модель нейронной сети прямого распространения (сети MLP), которая начинается с выравнивания данных, а затем проходит через два плотных слоя. Функция активации на последнем плотном слое (выходном слое)  отсутствует, что типично для задачи регрессии. В качестве потерь используется среднеквадратичную ошибку (MSE), так как, в отличие от MAE, функция MSE гладкая около нуля, что является полезным свойством для градиентного спуска. Показатель MAE будет отслеживаться в качестве метрики модели.

In [ ]:
inputs = tf.keras.Input(shape=(sequence_length, raw_data.shape[-1]))
x = tf.keras.layers.Flatten()(inputs)
x = tf.keras.layers.Dense(16, activation="relu")(x)
outputs = tf.keras.layers.Dense(1)(x)
model = tf.keras.Model(inputs, outputs)

model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
history = model.fit(train_dataset,
                    epochs=5,
                    validation_data=val_dataset,
                   ) 

print(f"MAE на тестовом наборе: {model.evaluate(test_dataset)[1]:.2f}")

In [ ]:
model.summary()

In [ ]:
loss = history.history["mae"]
val_loss = history.history["val_mae"]
epochs = range(1, len(loss) + 1)
plt.figure()
plt.plot(epochs, loss, "bo", label="MAE на обучающей выборке")
plt.plot(epochs, val_loss, "b", label="MAE на валидационной выборке")
plt.title("Ошибка MAE на обучающей и валидационной выборках")
plt.legend();

Результаты обучение нейронной сети свидетельствуют, что ошибка построенной модели на валидационном наборе (а также на тестовом наборе) превышает аналогичные ошибки для наивной модели прогнозирования температуры. Поэтому использование нейронной сети прямого распространения не позволяет получить модель прогнозирования приемлемого качества.

## Прогнозирование температуры при помощи сети RNN

Попробуем теперь построить модель прогнозирования температуры на основе простой сети RNN с одним скрытым слоем c 16 нейронами и выходным слоем из одного нейрона:

In [ ]:
inputs = tf.keras.Input(shape=(sequence_length, raw_data.shape[-1]))
x = tf.keras.layers.LSTM(16)(inputs) 
outputs = tf.keras.layers.Dense(1)(x)
model = tf.keras.Model(inputs, outputs)

model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
history = model.fit(train_dataset,
                    epochs=5,
                    validation_data=val_dataset,
                   ) 

print(f"Test MAE: {model.evaluate(test_dataset)[1]:.2f}")

In [ ]:
loss = history.history["mae"]
val_loss = history.history["val_mae"]
epochs = range(1, len(loss) + 1)
plt.figure()
plt.plot(epochs[1:], loss[1:], "bo", label="MAE на обучающей выборке")
plt.plot(epochs, val_loss, "b", label="MAE на валидационной выборке")
plt.title("Ошибка MAE на обучающей и валидационной выборках")
plt.legend()
plt.show()

Таким образом, применение сети RNN позволило улучшить показатели качества модели прогнозирования и получить модель с лучшим качеством прогноза, чем наивная модель прогнозирования. 

Почему получилось улучшить показатели качества модели по сравнению с моделью на основе сети MLP? Сети прямого распространения (сети MLP) не имеют памяти. Каждый элемент входных данных обрабатывается независимо, при этом, чтобы обработать последовательность или временной ряд точек данных, нужно отправить всю последовательность данных нейронной сети сразу, превратив ее в единую точку данных. 

Рекуррентная нейронная сеть (RNN) обрабатывает последовательность данных, перебирая элементы последовательности и поддерживая внутреннее состояние, содержащее информацию относительно того, что сеть видела до сих пор. По сути, сеть RNN — это тип нейронной сети с внутренним циклом.

Состояние сети RNN сбрасывается между обработкой двух разных независимых последовательностей (например, двух записей в пакете), так что по-прежнему  последовательность считается одной точкой данных. Но эта точка данных больше не обрабатывается за один шаг – скорее, сеть внутренне зацикливается на элементах последовательности.

## Загрузка котировок акций из Yahoo Finance


In [ ]:
#!pip install yfinance

In [ ]:
from pandas_datareader import data as pdr
import yfinance as yfin
import datetime as dt

yfin.pdr_override()

Загрузим ежедневные котировки акций компании Apple за период с 1 января 2016 по май 2022:

In [ ]:
aapl = pdr.get_data_yahoo('AAPL', 
                          start=dt.datetime(2016, 1, 1), 
                          end=dt.datetime(2022, 5, 31))
aapl.head()

In [ ]:
aapl.tail()

Извлечем первые записи с котировками за май 2022 г.:

In [ ]:
aapl.loc[pd.Timestamp('2022-05-01'):pd.Timestamp('2022-05-31')].head()

Извлечем первые записи с котировками за 2022 г.:

In [ ]:
aapl.loc['2022'].head()

Нарисуем график котировок закрытия:

In [ ]:
aapl['Close'].plot.line(grid=True,title='Котировки акций компании Apple в 2016-2022 гг.');

В качестве наиболее корректного показателя стоимости (цены) акции будем применять признак `Adj Close`. 

__Дневной доход__ по акции равен разности текущей цены акции и цены предыдущего дня. __Дневной убыток__ по акции равен разности цены предыдущего дня и текущей цены акции. __Дневная доходность__ по акции равна разности текущей цены акции и цены предыдущего дня, деленной на цену предыдущего дня. 

Произведем расчет дневных доходностей акции:

In [ ]:
d_close = aapl[['Adj Close']]
d_close

In [ ]:
d_pct_ch = d_close.pct_change()
d_pct_ch

In [ ]:
d_pct_ch2 = d_close / d_close.shift(1) - 1
d_pct_ch2

In [ ]:
d_pct_ch.describe()

In [ ]:
d_pct_ch.hist(bins=50);

### Задание на лабораторную работу №5

В соответствии с индивидуальным заданием, указанным в записной книжке команды, сделайте необходимые расчеты и постройте следующие визуализации:

1. При помощи модуля `pandas_datareader` считайте котировки указанной в индивидуальным задании акции за указанный период времени. 

2. Визуализируйте котировки акции (столбец `Adj Close`) за весь период на графике. Подпишите оси и рисунок.

3. Вычислите и визуализируйте заданный показатель акции в соответствии с индивидуальным заданием.

4. Сформируйте обучающую, тестовую и валидационные выборки для обучения нейронной сети в соответствии с индивидуальным заданием. 

5. Постройте нейронную сеть MLP с нормализующим слоем и одним плотным скрытым слоем из 16 нейронов для прогнозирования стоимости акции и обучите ее на обучающей выборке. Оцените качество прогнозирования при помощи заданного показателя качества на тестовой выборке.

6. Примените указанную в индивидуальном задании технику решения проблемы исчезающих градиентов и постройте нейронную сеть MLP с нормализующим слоем и тремя плотными скрытыми слоями из 16 нейронов для прогнозирования стоимости акции и обучите ее на обучающей выборке. Оцените качество прогнозирования при помощи заданного показателя качества для тестовой выборки.

7. Постройте рекуррентную нейронную сеть с нормализующим слоем и одним скрытым слоем LSTM из 16 нейронов для прогнозирования стоимости акции и обучите ее на обучающей выборке. Оцените качество прогнозирования при помощи заданного показателя качества на тестовой выборке.

8. Визуализируйте кривые обучения для трех построенных моделей на одном рисунке в зависимости от эпохи обучения, подписывая оси и рисунок и создавая легенду. Используйте для визуализации относительную ошибку (ошибку обучения, деленную на начальную ошибку на первой эпохе). 

9. Визуализируйте весь набор данных и прогнозы трех построенных моделей для обучающей и тестовой выборок на одном рисунке (ось X – даты, ось Y – стоимость акции), подписывая оси и рисунок и создавая легенду. 


